# simulation 作り直し

## プロセス


- オリジナルデータを作成
    - Home データの付与
    - 阪上データとの一致を確認
- 遷移確率を求める
- シミュレーションの作成



# 00.  ライブラリインストール

In [1]:
import pandas as pd
from tqdm import tqdm
import collections
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import numpy as np
from math import *
pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'japanize_matplotlib'

In [3]:
###### 変数定義　########

meshlist = [
    53393473,
    53393493,
    53394413,
    53393495,
    53393483,
    53393474,
    53393482,
    53393491,
    53393481,
    53394414,
    53393476,
    53393487,
    53394403,
    53393484,
    53393464,
    53393475,
    53393485,
    53393462,
    53393486,
    53393463,
    53393472,
    53393492,
    53394404,
    53393477,
    53393497,
    53394405,
    53393496,
    53394407,
    53393494,
    53394412,
    53393465,
    53394406,
    53394401,
    53394411,
    53393471,
    53394402,
    11111111
]


# 1.  合成人口データ確認
## 1-1 合成人工データの中身を確認

In [ ]:
#各合成人工データの中身を見る

pop_directory = '/Users/y-osamu/study/poi_sim/data/raw/population'

summary = []

for filename in os.listdir(pop_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(pop_directory, filename)
        df = pd.read_csv(file_path, index_col=0)
        num_rows, num_cols = df.shape
        summary.append({
            'ファイル名': filename,
            '行数': num_rows,
            '列数': num_cols
        })

summary_df = pd.DataFrame(summary)
print(summary_df)

                  ファイル名      行数  列数
0  2015_009_8_13208.csv  214281  24
1  2015_010_8_13208.csv  214281  24
2  2015_005_8_13208.csv  214281  24
3  2015_006_8_13208.csv  214281  24
4  2015_003_8_13208.csv  214281  24
5  2015_004_8_13208.csv  214281  24
6  2015_008_8_13208.csv  214281  24
7  2015_001_8_13208.csv  214281  24
8  2015_002_8_13208.csv  214281  24
9  2015_007_8_13208.csv  214281  24


In [8]:
# 合成人工データの確認

people_path = '/Users/y-osamu/study/poi_sim/data/processed/2015_001_8_13208_01.csv' # 阪上ファイル実験ファイル

people_df = pd.read_csv(people_path, index_col=0)

rename_dict = {
    "prefecture_code": "都道府県コード",
    "prefecture_name": "都道府県名",
    "city_code": "市区町村コード",
    "city_name": "市区町村名",
    "town_code": "町丁目コード",
    "town_name": "町丁目名",
    "latitude": "緯度",
    "longitude": "経度",
    "household_id": "世帯ID",
    "family_type_id": "世帯類型ID",
    "family_type": "世帯類型",
    "num_member": "世帯人数",
    "abnormal_household": "異常世帯フラグ",
    "person_id": "個人ID",
    "age": "年齢",
    "gender_id": "性別ID",
    "gender": "性別",
    "role_household_type_id": "世帯内役割ID",
    "role_household_type": "世帯内役割",
    "industry_type_id": "産業分類ID",
    "industry_type": "産業分類",
    "employment_type_id": "就業形態ID",
    "employment_type": "就業形態",
    "company_size_id": "企業規模ID",
    "company_size": "企業規模"
}

people_df = people_df.rename(columns=rename_dict)
people_df
# coordinate = df[['latitude','longitude']].to_numpy().tolist() #各エージェントの初期位置
# coordinate 

,都道府県コード,都道府県名,市区町村コード,市区町村名,町丁目コード,町丁目名,緯度,経度,世帯ID,世帯類型ID,世帯類型,世帯人数,異常世帯フラグ,個人ID,年齢,性別ID,性別,世帯内役割ID,世帯内役割,産業分類ID,産業分類,就業形態ID,就業形態,企業規模ID,企業規模
177,13,東京都,13208,調布市,13208003004,富士見町４丁目,35.662585,139.534513,18324823,0,単独世帯,1,0,40952264,18,0,男性,0,単独世帯（男性）,60.0,Ｆ 電気・ガス・熱供給・水道業,10.0,一般労働者,1000.0,1000人以上
181,13,東京都,13208,調布市,13208009002,染地２丁目,35.639386,139.549963,18324827,0,単独世帯,1,0,40952268,18,0,男性,0,単独世帯（男性）,130.0,Ｍ 宿泊業，飲食サービス業,20.0,短時間労働者,10.0,10～99人
187,13,東京都,13208,調布市,13208004003,下石原３丁目,35.649432,139.534088,18324833,0,単独世帯,1,0,40952274,18,0,男性,0,単独世帯（男性）,130.0,Ｍ 宿泊業，飲食サービス業,20.0,短時間労働者,1000.0,1000人以上
188,13,東京都,13208,調布市,13208004003,下石原３丁目,35.652872,139.530387,18324834,0,単独世帯,1,0,40952275,18,0,男性,0,単独世帯（男性）,90.0,Ｉ 卸売業，小売業,20.0,短時間労働者,100.0,100～999人
189,13,東京都,13208,調布市,13208001001,飛田給１丁目,35.663240,139.522585,18324835,0,単独世帯,1,0,40952276,18,0,男性,0,単独世帯（男性）,150.0,Ｏ 教育，学習支援業,20.0,短時間労働者,1000.0,1000人以上
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214267,13,東京都,13208,調布市,13208003003,富士見町３丁目,35.662458,139.541548,18429861,80,夫婦、子供とひとり親（夫の親）,8,0,41166354,24,0,男性,20,子供（男性）,120.0,Ｌ 学術研究，専門・技術サービス業,10.0,一般労働者,100.0,100～999人
214268,13,東京都,13208,調布市,13208003003,富士見町３丁目,35.662458,139.541548,18429861,80,夫婦、子供とひとり親（夫の親）,8,0,41166355,22,1,女性,21,子供（女性）,130.0,Ｍ 宿泊業，飲食サービス業,10.0,一般労働者,1000.0,1000人以上
214271,13,東京都,13208,調布市,13208003003,富士見町３丁目,35.662458,139.541548,18429861,80,夫婦、子供とひとり親（夫の親）,8,0,41166358,18,1,女性,21,子供（女性）,90.0,Ｉ 卸売業，小売業,10.0,一般労働者,1000.0,1000人以上
214273,13,東京都,13208,調布市,13208018002,緑ケ丘２丁目,35.670197,139.583564,18429862,80,夫婦、子供とひとり親（夫の親）,8,0,41166360,53,0,男性,10,夫・男親,50.0,Ｅ 製造業,10.0,一般労働者,1000.0,1000人以上


## 合成人工データ　調布市のデータ分布分析

In [4]:
tf = open("../data/trans_prob_mat_element.json", "r")
trans_prob_mat_element = json.load(tf)
trans_prob_mat_element

{'53393462_販売・卸': 0,
 '53393462_自動車・オートバイ・自転車・ドライブ': 1,
 '53393462_学校・趣味教室・図書館': 2,
 '53393462_医療・薬・保険衛生': 3,
 '53393462_建設・工事': 4,
 '53393462_官公庁・団体・福祉': 5,
 '53393462_製造・加工': 6,
 '53393462_飲食店': 7,
 '53393462_交通・運輸・倉庫': 8,
 '53393462_不動産・賃貸・展示場': 9,
 '53393462_スポーツ・趣味娯楽・レジャー': 10,
 '53393462_金融・保険・証券': 11,
 '53393462_電気・ガス・通信・放送・新聞': 12,
 '53393462_出版・印刷': 13,
 '53393462_旅行・観光・温泉・旅館・ホテル': 14,
 '53393462_農林・水産・鉱業': 15,
 '53393462_その他サービス': 16,
 '53393462_Home': 17,
 '53393462_OoR': 18,
 '53393462_Move': 19,
 '53393463_販売・卸': 20,
 '53393463_自動車・オートバイ・自転車・ドライブ': 21,
 '53393463_学校・趣味教室・図書館': 22,
 '53393463_医療・薬・保険衛生': 23,
 '53393463_建設・工事': 24,
 '53393463_官公庁・団体・福祉': 25,
 '53393463_製造・加工': 26,
 '53393463_飲食店': 27,
 '53393463_交通・運輸・倉庫': 28,
 '53393463_不動産・賃貸・展示場': 29,
 '53393463_スポーツ・趣味娯楽・レジャー': 30,
 '53393463_金融・保険・証券': 31,
 '53393463_電気・ガス・通信・放送・新聞': 32,
 '53393463_出版・印刷': 33,
 '53393463_旅行・観光・温泉・旅館・ホテル': 34,
 '53393463_農林・水産・鉱業': 35,
 '53393463_その他サービス': 36,
 '53393463_Home': 37,
 '533

In [5]:
SIZE = 10
N = 10 # エージェントの個数
START_HOUR = 6 # シミュレーション開始時間
END_HOUR = 11 # シミュレーション終了時間（台）



In [6]:
# 遷移確率行列のあるdirを読み込み
trans_prob_mat_dir = '/Users/osamu/study/sakagami/sakagami_processsing/pre/new'
trans_prob_mat_dir = '../data/new'

mat_files = sorted(glob.glob(os.path.join(trans_prob_mat_dir,'*.npy')))
matfile_dict = defaultdict(list)
for i in range(0,len(mat_files),4):
    sublist = mat_files[i:i+4]
    idx = int(i/4)+START_HOUR
    matfile_dict[idx] = sublist


In [7]:
# agoopデータに関する真値のあるdirを読み込み
true_val_dir = '/Users/osamu/study/sakagami/sakagami_processsing/true_data'
true_val_files = sorted(glob.glob(os.path.join(true_val_dir,'*.json')))
time_key = [i for i in range(START_HOUR,END_HOUR+1)]
true_val_files_dict = {k: v for k, v in zip(time_key, true_val_files)}
true_val_files

rue_val_files_dict = {}

for path in true_val_files:
    # ファイル名から "06" → 6 に変換
    filename = os.path.basename(path)  # 例: '06_fin.json'
    hour_str = filename.split('_')[0]  # → '06'
    hour = int(hour_str)               # → 6
    true_val_files_dict[hour] = path
    
true_val_files_dict[6]
with open(true_val_files[0]) as f:
    temp_true_data = json.load(f)
temp_true_data

{'53393462_販売・卸': 0,
 '53393462_自動車・オートバイ・自転車・ドライブ': 0,
 '53393462_学校・趣味教室・図書館': 1,
 '53393462_医療・薬・保険衛生': 0,
 '53393462_建設・工事': 0,
 '53393462_官公庁・団体・福祉': 0,
 '53393462_製造・加工': 0,
 '53393462_飲食店': 0,
 '53393462_交通・運輸・倉庫': 0,
 '53393462_不動産・賃貸・展示場': 0,
 '53393462_スポーツ・趣味娯楽・レジャー': 0,
 '53393462_金融・保険・証券': 0,
 '53393462_電気・ガス・通信・放送・新聞': 0,
 '53393462_出版・印刷': 0,
 '53393462_旅行・観光・温泉・旅館・ホテル': 0,
 '53393462_農林・水産・鉱業': 0,
 '53393462_その他サービス': 0,
 '53393462_Home': 1569,
 '53393462_OoR': 20,
 '53393462_Move': 542,
 '53393463_販売・卸': 0,
 '53393463_自動車・オートバイ・自転車・ドライブ': 0,
 '53393463_学校・趣味教室・図書館': 0,
 '53393463_医療・薬・保険衛生': 9,
 '53393463_建設・工事': 4,
 '53393463_官公庁・団体・福祉': 0,
 '53393463_製造・加工': 0,
 '53393463_飲食店': 0,
 '53393463_交通・運輸・倉庫': 0,
 '53393463_不動産・賃貸・展示場': 0,
 '53393463_スポーツ・趣味娯楽・レジャー': 0,
 '53393463_金融・保険・証券': 0,
 '53393463_電気・ガス・通信・放送・新聞': 0,
 '53393463_出版・印刷': 0,
 '53393463_旅行・観光・温泉・旅館・ホテル': 0,
 '53393463_農林・水産・鉱業': 0,
 '53393463_その他サービス': 1,
 '53393463_Home': 393,
 '53393463_OoR': 6,
 '533

In [8]:
#Agent class
class Agent:
    # コンストラクタ
    def __init__(self, lat, lng,p_idx):
        self.initial_lat = lat # エージェントの初期位置(lat)
        self.initial_lng = lng # エージェントの初期位置(lng)
        self.meshid = to_meshcode(lat,lng) # エージェントの初期位置（meshid）
        self.meshid_poi = str(self.meshid)+'_Home' # エージェントの初期位置(meshid_Home)
        self.p_idx = p_idx

    def set_meshid_poi(self,new_meshid_poi):
        self.meshid_poi = new_meshid_poi

    def get_meshid_poi(self):
        print(self.meshid_poi)

    # 移動の計算（遷移確率行列で移動させる）
    def calc_meshid_poi(self,meshid_poi_list,mat):
        # とりあえず属性は1つとして計算していく，後々属性を追加する際にはこのあたりの処理を変更する
        i = 0
        for meshid_poi in meshid_poi_list:
            row = mat[trans_prob_mat_element[meshid_poi]]
            result = np.random.choice(list(trans_prob_mat_element.keys()), SIZE,p=row)
            if i == 0:
                all_result = result
            else:
                all_result = np.append(all_result,result)
            i+=1
        return all_result
    
# lat,lngをmeshidに変換するメソッド
def to_meshcode2(lat, lng, EF=5):
    '''
    経度・緯度のndarrayを受け取り、
    メッシュコードのndarrayを返す
    EFは入力座標値の小数点以下の桁数-1を与える
    '''

    p, pmod = np.divmod(np.round(lat * 1.5, EF), 1)
    q, qmod = np.divmod(pmod * 8, 1)
    r, _ = np.divmod(qmod * 10 + 10**(-EF), 1)

    u, umod = np.divmod(lng - 100, 1)
    v, vmod = np.divmod(umod * 8, 1)
    w, _ = np.divmod(np.round(vmod, EF) * 10, 1)

    meshcode = (p * 10**6 + u * 10**4 +  # 1次メッシュ
                q * 10**3 + v * 10**2 +  # 2次メッシュ
                r * 10 + w )          # 3次メッシュ

    a = np.int32(11111111)

    if meshcode in meshlist:
        return meshcode.astype(np.int32)

    else:
        return a

    
def to_meshcode(lat, lng, EF=5):
    """
    経度・緯度のndarrayを受け取り、メッシュコードのndarrayを返す。
    """

    lat = np.round(lat * 1.5, EF)
    lng = lng - 100

    p = np.floor(lat)
    q = np.floor((lat - p) * 8)
    r = np.floor(((lat - p) * 8 - q) * 10)

    u = np.floor(lng)
    v = np.floor((lng - u) * 8)
    w = np.floor(((lng - u) * 8 - v) * 10)

    meshcode = (p * 10**6 + u * 10**4 +
                q * 10**3 + v * 10**2 +
                r * 10 + w).astype(np.int32)

    if meshlist is not None:
        # 配列対応: meshlistに含まれない要素を11111111に置き換え
        meshcode = np.where(np.isin(meshcode, meshlist), meshcode, 11111111)

    return meshcode

# to_meshcode(35.662585,139.534513)

# meshid_poiを.jsonで出力するメソッド
def write_meshid_poi(output_path,data):
    ext = os.path.basename(output_path).split('.')[1]

    if ext == 'json':
        json_file = open(output_path, 'w')
        json.dump(data,json_file,indent=4)
        json_file.close()

    if ext == 'npy':
        np.save(output_path, data)

def calc_cos_sim(mat1, mat2):
    v1, v2 = mat1.ravel(), mat2.ravel()
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def calc_cos_sim2(mat1,mat2):
    vector1 = mat1.flatten()
    vector2 = mat2.flatten()
    cosine_similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    return cosine_similarity

# ディレクトリを作るメソッド
def make_dir(output_dir):
    if not os.path.exists(output_dir):
        os.system('mkdir -p {}'.format(output_dir))

# meshid_poiを.jsonで出力するメソッド
def write_meshid_poi(output_path,data):
    ext = os.path.basename(output_path).split('.')[1]

    if ext == 'json':
        json_file = open(output_path, 'w')
        json.dump(data,json_file,indent=4)
        json_file.close()

    if ext == 'npy':
        np.save(output_path, data)

def read_meshid_poi(file_path):
    data = np.load(file_path)
    return data

def read_trans_prob_mat(file_path):
    data = np.load(file_path)
    return data

In [117]:
SIZE = 10
N = 10 # エージェントの個数
START_HOUR = 6 # シミュレーション開始時間
END_HOUR = 11 # シミュレーション終了時間（台）

FIRST_HOUR_FLAG=True
npz_file_num= 0

TARGET_MESHID = 53393473 # 比較対象とするMeshID

result_base_dir = '/Users/osamu/study/poi_sim/data/new'
result_base_dir = os.path.join(result_base_dir,people_path_noextension)

top3_dir = os.path.join(result_base_dir,'top3')
make_dir(top3_dir)
top3_dir

'/Users/osamu/study/poi_sim/data/new/2015_001_8_13208_01/top3'

In [10]:
currnet_mat_files = matfile_dict[hour]
print(f"[{hour}時] ファイル数: {len(currnet_mat_files)}")

[11時] ファイル数: 4


In [127]:
def simulate_1hour(hour):
    global FIRST_HOUR_FLAG
    global top3_pararel_list
    mat_idx = 0 # 遷移確率行列を参照する用
    SIZE = 10   # 世界の分岐数
    SIMULATION_NUM = 4 # シミュレーション数
    current_world_num = 1 # その時点での世界の数
    hour_count = 0 # 1時間ごとをカウントするもの
    POI_NUM = 20
    MESHID_NUM = 37

    person_dir = os.path.join(result_base_dir,'person_data',str(hour))
    output_dir = os.path.join(result_base_dir,'output_meshid_poi',str(hour))
    make_dir(person_dir)
    make_dir(output_dir)

    currnet_mat_files = matfile_dict[hour] # その時点での遷移確率行列を読み込む

    all_data=[]

    # 15分ごとに計算
    for min in range(SIMULATION_NUM):
        output_file = '_'.join([str((min+1)*15),'min','later'])
        output_file += '.json'
        output_path = os.path.join(output_dir,output_file)

        print('{}分後の世界'.format((min+1)*15))
        all_meshid_poi = []
        current_mat = np.load(currnet_mat_files[mat_idx])
        print('現在の遷移確率行列は',currnet_mat_files[mat_idx])
        mat_idx+=1

        # 並行世界を格納するためのdict(key:世界線,value:エージェントのmeshid_poi)
        meshid_poi_dict = collections.defaultdict(list)

        # ーーーーーーーーーーーーーー遷移確率行列で遷移させる（リストで1つずつ処理するもの）ーーーーーーーーーーーーーー
        print('計算中')
        for agent in tqdm(agentsA):
            person_file = '_'.join(['person',str(agent.p_idx+1)])
            person_file += '.npy'
            person_path = os.path.join(person_dir,person_file)

            if min == 0: # 15分時点
                temp_meshid_poi = [agent.meshid_poi]
                result_meshid_poi = agent.calc_meshid_poi(temp_meshid_poi,current_mat)
            else: # 0分以降
                temp_meshid_poi = read_meshid_poi(person_path)
                result_meshid_poi = agent.calc_meshid_poi(temp_meshid_poi,current_mat)
            write_meshid_poi(person_path,result_meshid_poi)
            all_meshid_poi.append(result_meshid_poi)


        # ーーーーーーーーーーーーーー遷移後のデータを整形するーーーーーーーーーーーーーー
        print('世界生成中')
        current_world_num *= SIZE
        # print('current_world_num',current_world_num)
        zero_list = [0] * len(temp_true_data)

        for j in tqdm(range(current_world_num)):
            world_meshid_poi = [all_meshid_poi[l][j] for l in range(N)]
            meshid_poi_dict[j+1] = world_meshid_poi

        if min != 0 and min % 3 == 0: # 各60分経過時点
            # 正解値はここに格納
            with open(true_val_files_dict[hour]) as f:
                true_data = json.load(f)

            print('現在の真値は',true_val_files_dict[hour])

            hour_count+=1
            true_mat = np.array(list(true_data.values())).reshape(POI_NUM,MESHID_NUM)
            print(true_mat.shape)

            # ーーーーーーーーーーーーーーコサイン類似度の計算ーーーーーーーーーーーーーー
            for world_num in meshid_poi_dict:
                world_meshid_poi = meshid_poi_dict[world_num]
                current_meshid_poi_dict = dict(zip(list(temp_true_data.keys()),zero_list))
                for meshid_poi in world_meshid_poi:
                    current_meshid = meshid_poi.split('_')[0]
                    current_meshid_poi_dict[meshid_poi]+=1

#                 # ー追加分ー
#                     if current_meshid == TARGET_MESHID:
#                         current_meshid_poi_dict[meshid_poi]+=1
#                     # with open('haruka.json', 'w') as json_file:
#                     #     json.dump(current_meshid_poi_dict, json_file)
                # ーーーー
                world_mat = np.array(list(current_meshid_poi_dict.values())).reshape(POI_NUM,MESHID_NUM)
                # 行列間の類似度計算
                cos_sim_val = calc_cos_sim(world_mat,true_mat)
                all_data.append((cos_sim_val,world_meshid_poi))

            # top10とbottom10のデータを表示
            top10 = sorted(all_data,reverse=True)[0:11]
            bottom10 = sorted(all_data,reverse=True)[-11:]
            print('top10')
            for data in top10:
                print(data[0])
            print('bottom10')
            for data in bottom10:
                print(data[0])

            # ーーーーーーーーーーーーーーTOP3を保持ーーーーーーーーーーーーーー
            all_data= sorted(all_data,reverse=False)
            top3 = all_data[0:3]
            output_top3_file = top3_dir+'/'+str(hour).zfill(2)+'_fin'
            top3_meshid_poi = [] # その世界における全員のmeshid_poiが保存
            for data in top3:
                top3_meshid_poi.append(np.array(data[1]))
            if FIRST_HOUR_FLAG: # 最初の1時間が経過,ここでは3つのグループができる
                np.savez(output_top3_file, top1=top3_meshid_poi[0],top2=top3_meshid_poi[1],top3=top3_meshid_poi[2])
                FIRST_HOUR_FLAG = False
            else: # それ以降，3つのグループ✖️3＝9つのグループができる
                for data in top3:
                    top3_pararel_list.append(data)
                if npz_file_num == 3: # 3グループ目の処理
                    all_data = sorted(top3_pararel_list,reverse=False)
                    top3 = all_data[0:3]
                    top3_meshid_poi = list()
                    for data in top3:
                        top3_meshid_poi.append(np.array(data[1]))
                    np.savez(output_top3_file, top1=top3_meshid_poi[0],top2=top3_meshid_poi[1],top3=top3_meshid_poi[2])

        write_meshid_poi(output_path,meshid_poi_dict)
        print('-----------------------------------')

# simulation_flle_num = 0

# for hour in range(START_HOUR,END_HOUR+1):
#     print('開始')
#     print('{}時台のシミュレーション'.format(hour))
#     # シミュレーション開始時は，初期値を合成人工データから持ってくる
#     top3_pararel_list = []

#     if hour == START_HOUR:
#         print('エージェント読み込み中')
#         print('-----------------------------------')
#         # エージェントの初期座標は合成人口データから持ってくる，6:00の処理
#         agentsA = np.array([Agent(coordinate[i][0],coordinate[i][1],i) for i in tqdm(range(N))])
#         simulate_1hour(hour)
#     # それ以降は，top3を使う
#     else:
#         simulation_files = sorted(glob.glob(os.path.join(top3_dir,'*')))
#         simulation_flle = simulation_files[simulation_flle_num]
#         # インクリメントすることで，1時間後にする
#         simulation_flle_num+=1
#         npz_file_num = 0
#         result_npz = np.load(simulation_flle)
#         for key in result_npz.files:
#             np_meshid_poi = result_npz[key]
#             npz_file_num+=1
#             for agent,result_meshid_poi in zip(agentsA,np_meshid_poi): # エージェントのmeshid_poiの初期化
#                 agent.set_meshid_poi(result_meshid_poi)
#             simulate_1hour(hour)


## シミュレーションを分解

- 各時間の挙動を確認
- Agents　クラスの実行結果を確認
- 各処理を確認
- データの持ち方を確認・変更

In [100]:
top3_meshid_poi[1]


array(['53393492_Home', '11111111_Move', '53393472_Home', '53393482_Home',
       '53393491_Home', '53393485_Home', '11111111_Move', '53393483_Home',
       '53393473_Home', '53393484_Home'], dtype='<U13')

In [130]:
START_HOUR = 6 # シミュレーション開始時間
END_HOUR = 11 # シミュレーション終了時間（台）
SIZE = 10
N = 10 # エージェントの個数
FIRST_HOUR_FLAG=True
npz_file_num= 0
TARGET_MESHID = 53393473 # 比較対象とするMeshID
simulation_flle_num = 0

start_time = time.time()

for hour in range(START_HOUR,END_HOUR+1):
    global FIRST_HOUR_FLAG
    global top3_pararel_list
    print('開始')
    print('{}時台のシミュレーション'.format(hour))
    # シミュレーション開始時は，初期値を合成人工データから持ってくる
    top3_pararel_list = []

    if hour == START_HOUR:
        print('エージェント読み込み中')
        print('-----------------------------------')
        # エージェントの初期座標は合成人口データから持ってくる，6:00の処理
        agentsA = np.array([Agent(coordinate[i][0],coordinate[i][1],i) for i in tqdm(range(N))])
        #simulate_1hour(hour)
        
        mat_idx = 0 # 遷移確率行列を参照する用
        SIZE = 10   # 世界の分岐数
        SIMULATION_NUM = 4 # シミュレーション数
        current_world_num = 1 # その時点での世界の数
        hour_count = 0 # 1時間ごとをカウントするもの
        POI_NUM = 20
        MESHID_NUM = 37

        person_dir = os.path.join(result_base_dir,'person_data',str(hour))
        output_dir = os.path.join(result_base_dir,'output_meshid_poi',str(hour))
        make_dir(person_dir)
        make_dir(output_dir)
        
        currnet_mat_files = matfile_dict[hour] # その時点での遷移確率行列を読み込む
        
        all_data=[]

        # 15分ごとに計算
        for min in range(SIMULATION_NUM):
            output_file = '_'.join([str((min+1)*15),'min','later'])
            output_file += '.json'
            output_path = os.path.join(output_dir,output_file)

            print('{}分後の世界'.format((min+1)*15))
            all_meshid_poi = []
            current_mat = np.load(currnet_mat_files[mat_idx])
            mat_idx+=1

            # 並行世界を格納するためのdict(key:世界線,value:エージェントのmeshid_poi)
            meshid_poi_dict = collections.defaultdict(list)

            for agent in tqdm(agentsA):
                person_file = '_'.join(['person',str(agent.p_idx+1)])
                person_file += '.npy'
                person_path = os.path.join(person_dir,person_file)

                if min == 0: # 15分時点
                    temp_meshid_poi = [agent.meshid_poi]
                    result_meshid_poi = agent.calc_meshid_poi(temp_meshid_poi,current_mat)
                else: # 0分以降
                    temp_meshid_poi = np.load(person_path)
                    result_meshid_poi = agent.calc_meshid_poi(temp_meshid_poi,current_mat)
                write_meshid_poi(person_path,result_meshid_poi)
                all_meshid_poi.append(result_meshid_poi)
                    

      # ーーーーーーーーーーーーーー遷移後のデータを整形するーーーーーーーーーーーーーー
            print('世界生成中')
            current_world_num *= SIZE
            print('current_world_num',current_world_num)
            zero_list = [0] * len(temp_true_data)

            
            for j in tqdm(range(current_world_num)):
                world_meshid_poi = [all_meshid_poi[l][j] for l in range(N)]
                meshid_poi_dict[j+1] = world_meshid_poi

            if min != 0 and min % 3 == 0: # 各60分経過時点
                with open(true_val_files_dict[hour]) as f:
                    true_data = json.load(f)

                print('現在の真値は',true_val_files_dict[hour])

                hour_count+=1
                true_mat = np.array(list(true_data.values())).reshape(POI_NUM,MESHID_NUM)
                print(true_mat.shape)

                # ーーーーーーーーーーーーーーコサイン類似度の計算ーーーーーーーーーーーーーー
                for world_num in meshid_poi_dict:
                    world_meshid_poi = meshid_poi_dict[world_num]
                    current_meshid_poi_dict = dict(zip(list(temp_true_data.keys()),zero_list))
                    for meshid_poi in world_meshid_poi:
                        current_meshid = meshid_poi.split('_')[0]
                        current_meshid_poi_dict[meshid_poi] += 1

#                     # ー追加分ー
#                         if current_meshid == TARGET_MESHID:
#                             current_meshid_poi_dict[meshid_poi]+=1
#                         # with open('haruka.json', 'w') as json_file:
#                         #     json.dump(current_meshid_poi_dict, json_file)
#                     # ーーーー
                    world_mat = np.array(list(current_meshid_poi_dict.values())).reshape(POI_NUM,MESHID_NUM)
#                     # 行列間の類似度計算
                    cos_sim_val = calc_cos_sim(world_mat,true_mat)
                    all_data.append((cos_sim_val,world_meshid_poi))

                # top10とbottom10のデータを表示
                top3 = sorted(all_data,reverse=True)[0:3]
                bottom3 = sorted(all_data,reverse=True)[-3:]
                #             print('top3')
                #             for data in top3:
                #                 print(data[0])
                #             print('bottom3')
                #             for data in bottom3:
                #                 print(data[0])

                       # ーーーーーーーーーーーーーーTOP3を保持ーーーーーーーーーーーーーー
                output_top3_file = top3_dir+'/'+str(hour).zfill(2)+'_fin'

                top3_meshid_poi = [] # その世界における全員のmeshid_poiが保存

                for i, data in enumerate(top3):
                    print(f"--- data {i+1} ---")
                    print("cos_sim:", data[0])      # 類似度スコア（float）
                    print("meshid_poi:", data[1])   # 各エージェントの位置（list）
                    top3_meshid_poi.append(np.array(data[1]))

                if FIRST_HOUR_FLAG: # 最初の1時間が経過,ここでは3つのグループができる
                    np.savez(output_top3_file, top1=top3_meshid_poi[0],top2=top3_meshid_poi[1],top3=top3_meshid_poi[2])
                    FIRST_HOUR_FLAG = False
                else: # それ以降，3つのグループ✖️3＝9つのグループができる
                    for data in top3:
                        top3_pararel_list.append(data)
                    if npz_file_num == 3: # 3グループ目の処理
                        all_data = sorted(top3_pararel_list,reverse=True)
                        top3 = all_data[0:3]
                        top3_meshid_poi = list()
                        for data in top3:
                            top3_meshid_poi.append(np.array(data[1]))
                        np.savez(output_top3_file, top1=top3_meshid_poi[0],top2=top3_meshid_poi[1],top3=top3_meshid_poi[2])

            write_meshid_poi(output_path,meshid_poi_dict)
        print('-----------------------------------')
#     else:
#         simulation_files = sorted(glob.glob(os.path.join(top3_dir,'*')))
#         simulation_flle = simulation_files[simulation_flle_num]
#         # インクリメントすることで，1時間後にする
#         simulation_flle_num+=1
#         npz_file_num = 0
#         result_npz = np.load(simulation_flle)
#         for key in result_npz.files:
#             np_meshid_poi = result_npz[key]
#             npz_file_num+=1
#             for agent,result_meshid_poi in zip(agentsA,np_meshid_poi):
#                 agent.set_meshid_poi(result_meshid_poi)
#             simulate_1hour(hour)
            
# print('終了')
end_time = time.time()
elapsed_time_seconds = end_time - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
print(f"処理にかかった時間: {elapsed_time_seconds:.2f} 秒")
print(f"処理にかかった時間: {elapsed_time_minutes:.2f} 分")

開始
6時台のシミュレーション
エージェント読み込み中
-----------------------------------


100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 4295.68it/s]


15分後の世界


100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 953.49it/s]


世界生成中
current_world_num 10


...

60分後の世界


100%|███████████████████████████████████████████| 10/10 [00:02<00:00,  4.74it/s]


世界生成中
current_world_num 10000


100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 231716.70it/s]


現在の真値は /Users/osamu/study/sakagami/sakagami_processsing/true_data/06_fin.json
(20, 37)
meshid_poi_dict 10000
--- data 1 ---
cos_sim: 0.734145548823984
meshid_poi: ['53393492_Home', '11111111_Move', '53393472_Home', '53393482_Home', '53393491_Home', '53393485_Home', '11111111_Move', '53393483_Home', '53393473_Home', '53393484_Home']
--- data 2 ---
cos_sim: 0.734145548823984
meshid_poi: ['53393492_Home', '11111111_Move', '53393472_Home', '53393482_Home', '53393491_Home', '53393485_Home', '11111111_Move', '53393483_Home', '53393473_Home', '53393484_Home']
--- data 3 ---
cos_sim: 0.734145548823984
meshid_poi: ['53393492_Home', '11111111_Move', '53393472_Home', '53393482_Home', '53393491_Home', '53393485_Home', '11111111_Move', '53393483_Home', '53393473_Home', '53393484_Home']
-----------------------------------
開始
7時台のシミュレーション
開始
8時台のシミュレーション
開始
9時台のシミュレーション
開始
10時台のシミュレーション
開始
11時台のシミュレーション
処理にかかった時間: 3.53 秒
処理にかかった時間: 0.06 分


In [140]:
world_mat = np.array(list(current_meshid_poi_dict.values())).reshape(POI_NUM,MESHID_NUM)
world_mat.shape

(20, 37)